# Utils

> Exporting to python module

In [ ]:
#| default_exp core.utils

In [ ]:
#|export
# standard
import logging
import os
from pathlib import Path
from typing import List
from configparser import ConfigParser

# 3rd party
import nbdev
from sklearn.utils import Bunch

## Logging

In [ ]:
#|export
def set_log_level(logger, log_level):
    logger.setLevel(log_level)
    ch = logging.StreamHandler()
    ch.setLevel(log_level)
    logger.addHandler(ch)

## cd_root

In [ ]:
#| export
def get_repo_root_folder(
    file_to_look_for_in_root_folder="settings.ini",
    max_parent_levels_to_traverse=10,
):
    """Gets root folder of repo where notebook is.

    It assumes that the root folder has a file called `file_to_look_for_in_root_folder`, which
    by default is `settings.ini`.
    """
    list_contents = os.listdir(".")
    traversed_parent_levels = 0
    while (
        file_to_look_for_in_root_folder not in list_contents
        and traversed_parent_levels < max_parent_levels_to_traverse
    ):
        traversed_parent_levels += 1
        os.chdir("..")
        list_contents = os.listdir(".")
    repo_root_folder = Path(".").resolve()
    return repo_root_folder

In [ ]:
#| export
def _cd_root_nbdev_impl():
    config = nbdev.config.get_config()
    os.chdir(config.config_path)

In [ ]:
#| export
def cd_root():
    repo_root_path = get_repo_root_folder()
    os.chdir(repo_root_path)

## Writing Tests

### Notebook structure 

> Utilities for building a dictionary with notebook structure. Useful for testing purposes.

#### make_nb_from_cell_list

In [ ]:
#| export
def make_nb_from_cell_list(cell_list: List[Bunch]):
    nb = Bunch(
        metadata=Bunch(
            kernelspec=Bunch(
                display_name="Python 3 (ipykernel)",
                language="python",
                name="python3",
            )
        ),
        cells=cell_list,
    )
    return nb


#### markdown_cell

In [ ]:
#| export
def markdown_cell(text: str):
    cell = Bunch(cell_type="markdown", metadata={}, source=text.splitlines())
    if len(cell.source[0].strip()) == 0:
        cell.source = cell.source[1:]
    cell.source = "\n".join(cell.source)
    return cell


#### code_cell

In [ ]:
#| export
def code_cell(text: str):
    cell = Bunch(
        cell_type="code",
        execution_count=None,
        metadata={},
        outputs=[],
        source=text.splitlines(),
    )
    if len(cell.source[0].strip()) == 0:
        cell.source = cell.source[1:]
    cell.source = "\n".join(cell.source)
    return cell

#### Example usage

In [ ]:
nb = make_nb_from_cell_list ([

markdown_cell ("""
## A minimal notebook
"""),

code_cell ("""
%%function one_plus_one --test
a=1+1
print (a)
"""),

])

assert nb == {'metadata': {'kernelspec': {'display_name': 'Python 3 (ipykernel)',
   'language': 'python',
   'name': 'python3'}},
 'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': '## A minimal notebook'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': '%%function one_plus_one --test\na=1+1\nprint (a)'}]}

### prepare_tests

In [ ]:
#| export
def prepare_tests ():
    pass

## get_config

In [ ]:
#| export
def get_config (path: str="settings.ini"):
    config = ConfigParser(delimiters=['='])
    config.read(path, encoding='utf-8')
    cfg = config['DEFAULT']
    cfg.config_path=Path (path).resolve()
    return cfg

### Example usage

In [ ]:
cd_root ()

cfg = get_config()

print (cfg.config_path)